# Murat Gonul's Capstone Project Study

## description of problem

In Turkey, coffie shops are becoming more and more popular and it is becoming a race for two brands that are penetrating to Turkish market for the last couple of years. Nero and Starbucks are having branches in all over Istanbul. Aim of the study is finding out best approach to get Nero ahead in the competition by using Foursquare data.

## how data might with that aim

Based on foursquare data, density of shops can be compared between two brands to see if any local market is missed. Additionally, data can show competition in terms of rating. Based on these inputs, a recommendation to Nero can be made to take this company ahead in the competition.

### initial data analysis
Blue dots are showing Nero and black dots are showing Starbucks branches in Taksim,Bakırköy,Zeytin Burnu, Beşiktaş and Bağdat Caddesi which are one of the most crowded veneus in Istanbul. Data can help with density of branches within main veneues.
For example, it can be seen that number of Nero branches are quite low compared to Starbucks in Bakırköy and Bağdat Caddesi. It can be wise to increase number of branches in there.


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

In [2]:
CLIENT_ID = 'TDLYPU2YXQQBECSDQ02CLIVWWVOAYG2MZB1THAR4N30FYJDI' # your Foursquare ID
CLIENT_SECRET = 'FLMVJU3X5EEKBQOHNWVIRVZUMBCWNOZGQY5VQ142UYYRTMWJ' # your Foursquare Secret
ACCESS_TOKEN = '2CRDYDMLUC0WO24ILPKQDIV3O3LYUTEOXU4STABDGUVQB02A' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TDLYPU2YXQQBECSDQ02CLIVWWVOAYG2MZB1THAR4N30FYJDI
CLIENT_SECRET:FLMVJU3X5EEKBQOHNWVIRVZUMBCWNOZGQY5VQ142UYYRTMWJ


In [3]:
addresses = ['taksim,istanbul','kadıköy,istanbul','bakırköy,istanbul','bağdat caddesi,istanbul','eminönü,istanbul','üsküdar,istanbul','zeytinburnu,istanbul']
brands = ['nero','starbucks']
radius = 1000
geolocator = Nominatim(user_agent="foursquare_agent")
latitude = []
longitude = []
adres = []
url = []
brandd = []
mini_lat = []
mini_long = []


for address in addresses:
        location = geolocator.geocode(address)
        mini_lat.append(location.latitude)
        mini_long.append(location.longitude)
        
df_mini = pd.DataFrame([mini_lat,mini_long,addresses])
df_mini = df_mini.transpose()

df_mini.columns = ['Latitude','Longitude','Address']
df_mini


,Latitude,Longitude,Address
0,41.0381,28.9855,"taksim,istanbul"
1,40.9916,29.027,"kadıköy,istanbul"
2,40.9835,28.868,"bakırköy,istanbul"
3,40.952,29.097,"bağdat caddesi,istanbul"
4,41.0141,28.9759,"eminönü,istanbul"
5,41.0249,29.0175,"üsküdar,istanbul"
6,40.9881,28.9036,"zeytinburnu,istanbul"


In [4]:
for address in addresses:
    for brand in brands:
        location = geolocator.geocode(address)
        latitude.append(location.latitude)
        longitude.append(location.longitude)
        adres.append(address.split(',',1)[0])
        search_query = brand
        brandd.append(brand)
        url.append('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, location.latitude, location.longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT))


df_loc = pd.DataFrame([latitude,longitude,adres,brandd,url])
df_loc = df_loc.transpose()

df_loc.columns = ['Latitude','Longitude','Address','Brand','URL']
df_loc

,Latitude,Longitude,Address,Brand,URL
0,41.0381,28.9855,taksim,nero,https://api.foursquare.com/v2/venues/search?cl...
1,41.0381,28.9855,taksim,starbucks,https://api.foursquare.com/v2/venues/search?cl...
2,40.9916,29.027,kadıköy,nero,https://api.foursquare.com/v2/venues/search?cl...
3,40.9916,29.027,kadıköy,starbucks,https://api.foursquare.com/v2/venues/search?cl...
4,40.9835,28.868,bakırköy,nero,https://api.foursquare.com/v2/venues/search?cl...
5,40.9835,28.868,bakırköy,starbucks,https://api.foursquare.com/v2/venues/search?cl...
6,40.952,29.097,bağdat caddesi,nero,https://api.foursquare.com/v2/venues/search?cl...
7,40.952,29.097,bağdat caddesi,starbucks,https://api.foursquare.com/v2/venues/search?cl...
8,41.0141,28.9759,eminönü,nero,https://api.foursquare.com/v2/venues/search?cl...
9,41.0141,28.9759,eminönü,starbucks,https://api.foursquare.com/v2/venues/search?cl...


In [5]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
def URL_get(info):
    results = requests.get(info).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    return dataframe_filtered

In [7]:
big_list_address = []
big_list_name = []
big_list_lat = []
big_list_long = []
big_list_url = []
big_list_brand = []

for i,j in enumerate(df_loc['URL']):
    getting = URL_get(j)
    for lat, lng, name in zip(getting.lat, getting.lng, getting.name):
        big_list_address.append(df_loc['Address'].iloc[i])
        big_list_brand.append(df_loc['Brand'].iloc[i])
        big_list_url.append(df_loc['URL'].iloc[i])
        big_list_lat.append(lat)
        big_list_long.append(lng)
        big_list_name.append(name)

In [8]:
df_loc_big = pd.DataFrame([big_list_lat,big_list_long,big_list_name,big_list_address,big_list_brand,big_list_url])
df_loc_big = df_loc_big.transpose()

df_loc_big.columns = ['Latitude','Longitude','Name','Address','Brand','URL']
df_loc_big.head()

,Latitude,Longitude,Name,Address,Brand,URL
0,41.0339,28.9769,Caffè Nero,taksim,nero,https://api.foursquare.com/v2/venues/search?cl...
1,41.0359,28.9817,Nero,taksim,nero,https://api.foursquare.com/v2/venues/search?cl...
2,41.0408,28.985,Cafe Nero,taksim,nero,https://api.foursquare.com/v2/venues/search?cl...
3,41.0359,28.9816,NERO Life,taksim,nero,https://api.foursquare.com/v2/venues/search?cl...
4,41.0318,28.9896,Nero Plastik Ambalaj San. ve Tic. Ltd. Şti.,taksim,nero,https://api.foursquare.com/v2/venues/search?cl...


In [9]:
latitude = df_loc_big['Latitude'].mean()
longitude = df_loc_big['Longitude'].mean()

In [10]:
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the Conrad Hotel

for lat, lng, label in zip(df_mini.Latitude, df_mini.Longitude, df_mini.Address):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        color='red',
        popup=label,
        fill = True,
        fill_color = 'red',
        fill_opacity = 0.6
    ).add_to(venues_map)
    
    
# set color scheme for the clusters
x = np.arange(len(brands))
ys = [i + x + (i*x)**2 for i in range(len(brands))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


for lat, lng, label, brand in zip(df_loc_big.Latitude, df_loc_big.Longitude, df_loc_big.Address, df_loc_big.Brand):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color = 'blue' if brand == 'nero' else 'black',
        popup = brand + ' ' + label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map